# TP3

## 0. import packages

In [1]:
import org.apache.spark.sql.DataFrame

import org.apache.spark.ml.feature.RegexTokenizer
import org.apache.spark.ml.feature.StopWordsRemover
import org.apache.spark.ml.feature.{CountVectorizer, CountVectorizerModel}
import org.apache.spark.ml.feature.IDF
import org.apache.spark.ml.feature.StringIndexer
import org.apache.spark.ml.feature.OneHotEncoderEstimator
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.feature.Bucketizer
import org.apache.spark.ml.feature.StandardScaler

import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.classification.DecisionTreeClassifier

import org.apache.spark.ml.{Pipeline, PipelineModel}

import org.apache.spark.ml.tuning.{ParamGridBuilder, TrainValidationSplit}
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator

Intitializing Scala interpreter ...

Spark Web UI available at http://172.21.129.6:4041
SparkContext available as 'sc' (version = 2.3.4, master = local[*], app id = local-1575748940783)
SparkSession available as 'spark'


import org.apache.spark.sql.DataFrame
import org.apache.spark.ml.feature.RegexTokenizer
import org.apache.spark.ml.feature.StopWordsRemover
import org.apache.spark.ml.feature.{CountVectorizer, CountVectorizerModel}
import org.apache.spark.ml.feature.IDF
import org.apache.spark.ml.feature.StringIndexer
import org.apache.spark.ml.feature.OneHotEncoderEstimator
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.feature.Bucketizer
import org.apache.spark.ml.feature.StandardScaler
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.classification.DecisionTreeClassifier
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.tuning.{ParamGridBuilder, TrainValidationSplit}
import org.apache.spark.ml.evaluati...

## Load DataFrame

In [2]:
val path_to_data : String = "ressources/"

//val df: DataFrame = spark.read.parquet(path_to_data + "prepared_trainingset")
// From TP2
val df: DataFrame = spark.read.parquet(path_to_data + "preprocessed")

println("From TP2")
println(s"\tNombre de lignes : ${df.count}")
println(s"\tNombre de colonnes : ${df.columns.length}")

val preprocessed: DataFrame  = df
    .filter($"days_campaign" =!= -1)
    .filter($"hours_prepa" =!= -1)
    .filter($"goal" =!= -1)
    .filter($"country2" =!= "Unknown")
    .filter($"currency2" =!= "Unknown")

println("\nAfter additional cleaning")
println(s"\tNombre de lignes : ${preprocessed.count}")
println(s"\tNombre de colonnes : ${preprocessed.columns.length}")

preprocessed.show(10)

From TP2
	Nombre de lignes : 108129
	Nombre de colonnes : 11

After additional cleaning
	Nombre de lignes : 107615
	Nombre de colonnes : 11
+--------------+--------------------+--------------------+------+--------------------+------------+--------+---------+-------------+-----------+--------------------+
|    project_id|                name|                desc|  goal|            keywords|final_status|country2|currency2|days_campaign|hours_prepa|                text|
+--------------+--------------------+--------------------+------+--------------------+------------+--------+---------+-------------+-----------+--------------------+
| kkst471421639|american options ...|looking to create...|100000|american-options-...|           0|      US|      USD|           30|    140.125|american options ...|
|kkst1098019088|iheadbones bone c...|wireless bluetoot...| 20000|iheadbones-bone-c...|           0|      US|      USD|           30|   18642.51|iheadbones bone c...|
|kkst1719475563| the fridge ma

path_to_data: String = ressources/
df: org.apache.spark.sql.DataFrame = [project_id: string, name: string ... 9 more fields]
preprocessed: org.apache.spark.sql.DataFrame = [project_id: string, name: string ... 9 more fields]


In [3]:
preprocessed.printSchema()

root
 |-- project_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- desc: string (nullable = true)
 |-- goal: integer (nullable = true)
 |-- keywords: string (nullable = true)
 |-- final_status: integer (nullable = true)
 |-- country2: string (nullable = true)
 |-- currency2: string (nullable = true)
 |-- days_campaign: integer (nullable = true)
 |-- hours_prepa: double (nullable = true)
 |-- text: string (nullable = true)



In [4]:
import scala.math.log1p

def goalLog(x: Double): Double = {
    val x_out = log1p(x)
    x_out
}

val goalLogUdf = udf(goalLog _)

val preprocessed_2: DataFrame = preprocessed
  .withColumn("goal", $"goal".cast("Double"))
  .withColumn("goal2", goalLogUdf($"goal"))


preprocessed_2.select("goal", "goal2").show(10)

+--------+------------------+
|    goal|             goal2|
+--------+------------------+
|100000.0| 11.51293546492023|
| 20000.0|  9.90353755128617|
|   700.0|  6.55250788703459|
| 12800.0|  9.45727857185611|
|  1500.0| 7.313886831633462|
| 60000.0|11.002116507732017|
| 10000.0| 9.210440366976517|
|  1000.0|  6.90875477931522|
|  1000.0|  6.90875477931522|
|145000.0|11.884495917930655|
+--------+------------------+
only showing top 10 rows



import scala.math.log1p
goalLog: (x: Double)Double
goalLogUdf: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(<function1>,DoubleType,Some(List(DoubleType)))
preprocessed_2: org.apache.spark.sql.DataFrame = [project_id: string, name: string ... 10 more fields]


In [5]:
preprocessed_2.printSchema()

root
 |-- project_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- desc: string (nullable = true)
 |-- goal: double (nullable = true)
 |-- keywords: string (nullable = true)
 |-- final_status: integer (nullable = true)
 |-- country2: string (nullable = true)
 |-- currency2: string (nullable = true)
 |-- days_campaign: integer (nullable = true)
 |-- hours_prepa: double (nullable = true)
 |-- text: string (nullable = true)
 |-- goal2: double (nullable = true)



## 2. Transform text into a feature vector

In [6]:
val tokenizer = new RegexTokenizer()
  .setPattern("\\W+")
  .setGaps(true)
  .setInputCol("text")
  .setOutputCol("tokens")

val tokenized = tokenizer.transform(preprocessed_2)

tokenized.select("tokens").show(10)

+--------------------+
|              tokens|
+--------------------+
|[american, option...|
|[iheadbones, bone...|
|[the, fridge, mag...|
|[support, new, me...|
|[can, t, a, psych...|
|[fragmented, fate...|
|[transport, suspe...|
|[the, secret, lif...|
|[cc, survival, de...|
|[the, best, prote...|
+--------------------+
only showing top 10 rows



tokenizer: org.apache.spark.ml.feature.RegexTokenizer = regexTok_ffeff5c1581b
tokenized: org.apache.spark.sql.DataFrame = [project_id: string, name: string ... 11 more fields]


## 3. Remove stop words

In [7]:
val remover = new StopWordsRemover()
  .setInputCol(tokenizer.getOutputCol)
  .setOutputCol("filtered")

StopWordsRemover.loadDefaultStopWords("english")

val filtered = remover.transform(tokenized)

filtered.select("tokens", "filtered").show(10)

+--------------------+--------------------+
|              tokens|            filtered|
+--------------------+--------------------+
|[american, option...|[american, option...|
|[iheadbones, bone...|[iheadbones, bone...|
|[the, fridge, mag...|[fridge, magazine...|
|[support, new, me...|[support, new, me...|
|[can, t, a, psych...|[psychological, h...|
|[fragmented, fate...|[fragmented, fate...|
|[transport, suspe...|[transport, suspe...|
|[the, secret, lif...|[secret, life, mo...|
|[cc, survival, de...|[cc, survival, de...|
|[the, best, prote...|[best, protein, d...|
+--------------------+--------------------+
only showing top 10 rows



remover: org.apache.spark.ml.feature.StopWordsRemover = stopWords_ee25c6ea29c4
filtered: org.apache.spark.sql.DataFrame = [project_id: string, name: string ... 12 more fields]


## 4. Compute *TF* part

In [8]:
// fit a CountVectorizerModel from the corpus
val countVectorizer: CountVectorizer = new CountVectorizer()
  .setInputCol(remover.getOutputCol)
  .setOutputCol("tf")
//  .setMinDF(55)

val tfModel = countVectorizer.fit(filtered)

val featurized = tfModel.transform(filtered)

featurized.select("filtered", "tf").show(20)

+--------------------+--------------------+
|            filtered|                  tf|
+--------------------+--------------------+
|[american, option...|(97856,[33,43,91,...|
|[iheadbones, bone...|(97856,[701,874,8...|
|[fridge, magazine...|(97856,[0,7,87,30...|
|[support, new, me...|(97856,[0,60,86,1...|
|[psychological, h...|(97856,[3,10,13,6...|
|[fragmented, fate...|(97856,[88,114,15...|
|[transport, suspe...|(97856,[2,19,44,6...|
|[secret, life, mo...|(97856,[11,19,31,...|
|[cc, survival, de...|(97856,[9,118,286...|
|[best, protein, d...|(97856,[50,79,218...|
|[paradise, falls,...|(97856,[9,10,23,1...|
|[chalet, woodshop...|(97856,[7,33,185,...|
|[vagabond, mobile...|(97856,[4,14,105,...|
|[southern, shakes...|(97856,[50,103,11...|
|[leviathan, monta...|(97856,[32,42,110...|
|[candle, tray, ha...|(97856,[48,90,150...|
|[sun, skin, missi...|(97856,[110,571,8...|
|[7sonic, debut, s...|(97856,[1,18,42,4...|
|[hades, pit, fema...|(97856,[3,52,53,6...|
|[fitness, refiner...|(97856,[33

countVectorizer: org.apache.spark.ml.feature.CountVectorizer = cntVec_7a4acd38a546
tfModel: org.apache.spark.ml.feature.CountVectorizerModel = cntVec_7a4acd38a546
featurized: org.apache.spark.sql.DataFrame = [project_id: string, name: string ... 13 more fields]


## 5. Compute *IDF* part

In [9]:
val idf = new IDF()
    .setInputCol(countVectorizer.getOutputCol)
    .setOutputCol("tfidf")

val idfModel = idf.fit(featurized)

val rescaled = idfModel.transform(featurized)

rescaled.select("tf", "tfidf").show(false)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|tf                                                                                                                                                                                                |tfidf                                                                                                    

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
only showing top 20 rows



idf: org.apache.spark.ml.feature.IDF = idf_40ec64006757
idfModel: org.apache.spark.ml.feature.IDFModel = idf_40ec64006757
rescaled: org.apache.spark.sql.DataFrame = [project_id: string, name: string ... 14 more fields]


In [11]:
rescaled.printSchema()

root
 |-- project_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- desc: string (nullable = true)
 |-- goal: double (nullable = true)
 |-- keywords: string (nullable = true)
 |-- final_status: integer (nullable = true)
 |-- country2: string (nullable = true)
 |-- currency2: string (nullable = true)
 |-- days_campaign: integer (nullable = true)
 |-- hours_prepa: double (nullable = true)
 |-- text: string (nullable = true)
 |-- goal2: double (nullable = true)
 |-- tokens: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- filtered: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- tf: vector (nullable = true)
 |-- tfidf: vector (nullable = true)



## 6. Convert categorial variables into numerical variables

In [12]:
val countryIndexer = new StringIndexer()
  .setInputCol("country2")
  .setOutputCol("country_indexed")
  .setHandleInvalid("keep")

val countryIndexed = countryIndexer.fit(rescaled).transform(rescaled)

countryIndexed.select("country2", "country_indexed").distinct().show()

+--------+---------------+
|country2|country_indexed|
+--------+---------------+
|      AU|            3.0|
|      DK|            7.0|
|      GB|            1.0|
|      NZ|            5.0|
|      NO|            8.0|
|      NL|            4.0|
|      DE|           10.0|
|      CA|            2.0|
|      US|            0.0|
|      SE|            6.0|
|      IE|            9.0|
+--------+---------------+



countryIndexer: org.apache.spark.ml.feature.StringIndexer = strIdx_c6c7c97a3cc5
countryIndexed: org.apache.spark.sql.DataFrame = [project_id: string, name: string ... 15 more fields]


In [13]:
val currencyIndexer = new StringIndexer()
  .setInputCol("currency2")
  .setOutputCol("currency_indexed")
  .setHandleInvalid("keep")

val currencyIndexed = currencyIndexer.fit(countryIndexed).transform(countryIndexed)
currencyIndexed.select("currency2", "currency_indexed").show()

+---------+----------------+
|currency2|currency_indexed|
+---------+----------------+
|      USD|             0.0|
|      USD|             0.0|
|      USD|             0.0|
|      USD|             0.0|
|      USD|             0.0|
|      USD|             0.0|
|      EUR|             4.0|
|      GBP|             1.0|
|      GBP|             1.0|
|      USD|             0.0|
|      GBP|             1.0|
|      USD|             0.0|
|      GBP|             1.0|
|      USD|             0.0|
|      USD|             0.0|
|      USD|             0.0|
|      USD|             0.0|
|      USD|             0.0|
|      GBP|             1.0|
|      GBP|             1.0|
+---------+----------------+
only showing top 20 rows



currencyIndexer: org.apache.spark.ml.feature.StringIndexer = strIdx_4fc8b737bec5
currencyIndexed: org.apache.spark.sql.DataFrame = [project_id: string, name: string ... 16 more fields]


In [14]:
val encoder = new OneHotEncoderEstimator()
  .setInputCols(Array("country_indexed", "currency_indexed"))
  .setOutputCols(Array("country_onehot", "currency_onehot"))

val model = encoder.fit(currencyIndexed)

val encoded = model.transform(currencyIndexed)

encoded.select("country_indexed", "currency_indexed",
               "country_onehot", "currency_onehot").show(false)

+---------------+----------------+--------------+---------------+
|country_indexed|currency_indexed|country_onehot|currency_onehot|
+---------------+----------------+--------------+---------------+
|0.0            |0.0             |(11,[0],[1.0])|(9,[0],[1.0])  |
|0.0            |0.0             |(11,[0],[1.0])|(9,[0],[1.0])  |
|0.0            |0.0             |(11,[0],[1.0])|(9,[0],[1.0])  |
|0.0            |0.0             |(11,[0],[1.0])|(9,[0],[1.0])  |
|0.0            |0.0             |(11,[0],[1.0])|(9,[0],[1.0])  |
|0.0            |0.0             |(11,[0],[1.0])|(9,[0],[1.0])  |
|4.0            |4.0             |(11,[4],[1.0])|(9,[4],[1.0])  |
|1.0            |1.0             |(11,[1],[1.0])|(9,[1],[1.0])  |
|1.0            |1.0             |(11,[1],[1.0])|(9,[1],[1.0])  |
|0.0            |0.0             |(11,[0],[1.0])|(9,[0],[1.0])  |
|1.0            |1.0             |(11,[1],[1.0])|(9,[1],[1.0])  |
|0.0            |0.0             |(11,[0],[1.0])|(9,[0],[1.0])  |
|1.0      

encoder: org.apache.spark.ml.feature.OneHotEncoderEstimator = oneHotEncoder_5409e3e8c291
model: org.apache.spark.ml.feature.OneHotEncoderModel = oneHotEncoder_5409e3e8c291
encoded: org.apache.spark.sql.DataFrame = [project_id: string, name: string ... 18 more fields]


In [15]:
encoded.printSchema()

root
 |-- project_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- desc: string (nullable = true)
 |-- goal: double (nullable = true)
 |-- keywords: string (nullable = true)
 |-- final_status: integer (nullable = true)
 |-- country2: string (nullable = true)
 |-- currency2: string (nullable = true)
 |-- days_campaign: integer (nullable = true)
 |-- hours_prepa: double (nullable = true)
 |-- text: string (nullable = true)
 |-- goal2: double (nullable = true)
 |-- tokens: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- filtered: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- tf: vector (nullable = true)
 |-- tfidf: vector (nullable = true)
 |-- country_indexed: double (nullable = false)
 |-- currency_indexed: double (nullable = false)
 |-- country_onehot: vector (nullable = true)
 |-- currency_onehot: vector (nullable = true)



## Mettre les données sous une forme utilisable par Spark.ML

### 9 : assembler tous les features en un unique vecteur

In [16]:
val assembler = new VectorAssembler()
  .setInputCols(Array("tfidf", "days_campaign", "hours_prepa", "goal2", "country_onehot", "currency_onehot"))
  .setOutputCol("features")

val assembled = assembler.transform(encoded)

assembled.select("features").show(false)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|features                                                                                                                                                                                                                                                                                                                                                                                                                                

assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_969e9f958621
assembled: org.apache.spark.sql.DataFrame = [project_id: string, name: string ... 19 more fields]


## Stage 10 : créer/instancier le modèle de classification

In [17]:
val lr = new LogisticRegression()
  .setElasticNetParam(0.0)
  .setFitIntercept(true)
  .setFeaturesCol("features")
  .setLabelCol("final_status")
  .setStandardization(true)
  .setPredictionCol("predictions")
  .setRawPredictionCol("raw_predictions")
  .setThresholds(Array(0.7, 0.3))
  .setTol(1.0e-6)
  .setMaxIter(30)

lr: org.apache.spark.ml.classification.LogisticRegression = logreg_08747a4c3caa


## Pipeline

In [18]:
val pipeline = new Pipeline()
  .setStages(Array(tokenizer, remover, countVectorizer, idf,
                   countryIndexer, currencyIndexer, encoder, assembler, lr))

pipeline: org.apache.spark.ml.Pipeline = pipeline_28d1f4013846


## Training, test and backup of the model

### Split of the data in 2 sets: `training` and `test`

We split the data in 2 sets: 90 % of the data for `training` and 10 % for `testing`.
Note that we use the initial dataframe `preprocessed`.

In [21]:
val Array(training, test) = preprocessed_2.randomSplit(Array(0.9, 0.1), 13)

training: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [project_id: string, name: string ... 10 more fields]
test: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [project_id: string, name: string ... 10 more fields]


### Training of the model

In [22]:
// create a model from training data
val model_one = pipeline.fit(training)

model_one: org.apache.spark.ml.PipelineModel = pipeline_28d1f4013846


### Evaluation of the model / test

In [23]:
// Make predictions from test data
val dfWithSimplePredictions = model_one.transform(test)
    .select("features", "final_status", "predictions")

dfWithSimplePredictions.groupBy("final_status", "predictions").count.show()

+------------+-----------+-----+
|final_status|predictions|count|
+------------+-----------+-----+
|           1|        0.0| 1832|
|           0|        0.0| 5043|
|           1|        1.0| 1636|
|           0|        1.0| 2244|
+------------+-----------+-----+



dfWithSimplePredictions: org.apache.spark.sql.DataFrame = [features: vector, final_status: int ... 1 more field]


In [24]:
// We use f1-score to compare models
val evaluator = new MulticlassClassificationEvaluator()
    .setLabelCol("final_status")
    .setPredictionCol("predictions")
    .setMetricName("f1")

val f1_score = evaluator.evaluate(dfWithSimplePredictions)

printf("The f1 score on test set is : %.3f\n", f1_score)

The f1 score on test set is : 0.626


evaluator: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_1ddce334530d
f1_score: Double = 0.6261255824765122


## Tuning of hyper-parameters of the model

In [25]:
//Entraînement du classifieur et réglage des hyper-paramètres de l’algorithme
/** k) Préparer la grid-search pour satisfaire les conditions explicitées ci-dessus
 *    puis lancer la grid-search sur le dataset “training” préparé précédemment.  */
val paramGrid = new ParamGridBuilder()
    .addGrid(lr.regParam, Array(10e-8, 10e-6, 10e-4, 10e-2))
    .addGrid(countVectorizer.minDF, Array[Double](55.0, 75.0, 95.0))
    .build()

val trainValidationSplit = new TrainValidationSplit()
    .setEstimator(pipeline)
    .setEvaluator(evaluator)
    .setEstimatorParamMaps(paramGrid)
    .setTrainRatio(0.7) // 70% of the data will be used for training and the remaining 30% for validation.

paramGrid: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	cntVec_7a4acd38a546-minDF: 55.0,
	logreg_08747a4c3caa-regParam: 1.0E-7
}, {
	cntVec_7a4acd38a546-minDF: 55.0,
	logreg_08747a4c3caa-regParam: 1.0E-5
}, {
	cntVec_7a4acd38a546-minDF: 55.0,
	logreg_08747a4c3caa-regParam: 0.001
}, {
	cntVec_7a4acd38a546-minDF: 55.0,
	logreg_08747a4c3caa-regParam: 0.1
}, {
	cntVec_7a4acd38a546-minDF: 75.0,
	logreg_08747a4c3caa-regParam: 1.0E-7
}, {
	cntVec_7a4acd38a546-minDF: 75.0,
	logreg_08747a4c3caa-regParam: 1.0E-5
}, {
	cntVec_7a4acd38a546-minDF: 75.0,
	logreg_08747a4c3caa-regParam: 0.001
}, {
	cntVec_7a4acd38a546-minDF: 75.0,
	logreg_08747a4c3caa-regParam: 0.1
}, {
	cntVec_7a4acd38a546-minDF: 95.0,
	logreg_08747a4c3caa-regParam: 1.0E-7
}, {
	cntVec_7a4acd38a546-minDF: 95.0,
	logreg_08747a4...

In [26]:
// Fit the pipeline to training documents.
val tvs_model = trainValidationSplit.fit(training)

tvs_model: org.apache.spark.ml.tuning.TrainValidationSplitModel = tvs_c397b1fd4a34


In [27]:
val predictions = tvs_model
    .transform(test)
    .select("features", "final_status", "predictions")

predictions.groupBy("final_status", "predictions").count.show()

+------------+-----------+-----+
|final_status|predictions|count|
+------------+-----------+-----+
|           1|        0.0|  946|
|           0|        0.0| 4590|
|           1|        1.0| 2522|
|           0|        1.0| 2697|
+------------+-----------+-----+



predictions: org.apache.spark.sql.DataFrame = [features: vector, final_status: int ... 1 more field]


In [28]:
val f1_score = evaluator.evaluate(predictions)

printf("The f1 score on test set is : %.3f\n", f1_score)

The f1 score on test set is : 0.672


f1_score: Double = 0.6722848061132849


In [ ]:
val bestModel = tvs_model.bestModel

### save the model on disk

In [ ]:
// Now we can save the fitted pipeline to disk
tvs_model.write.overwrite().save("save/spark-logistic-regression-model")

### Decision tree

In [ ]:
import org.apache.spark.ml.classification.RandomForestClassifier

In [ ]:
// Train a RandomForest model.
val rf = new RandomForestClassifier()
  .setFeaturesCol("features")
  .setLabelCol("final_status")
  .setNumTrees(10)
  .setPredictionCol("predictions")
  .setRawPredictionCol("raw_predictions")

val pipeline_2 = new Pipeline()
  .setStages(Array(tokenizer, remover, countVectorizer, idf,
                   countryIndexer, currencyIndexer, encoder, assembler, rf))

// create a model from training data
val model_2 = pipeline_2.fit(training)

In [ ]:
val paramGrid_2 = new ParamGridBuilder()
    .addGrid(rf.maxDepth, Array(5, 10, 15))
    .addGrid(rf.numTrees, Array(5, 10, 15, 20))
    .addGrid(countVectorizer.minDF, Array[Double](55.0, 75.0, 95.0))
    .build()

val trainValidationSplit_2 = new TrainValidationSplit()
    .setEstimator(pipeline_2)
    .setEvaluator(evaluator)
    .setEstimatorParamMaps(paramGrid_2)
    .setTrainRatio(0.7) // 70% of the data will be used for training and the remaining 30% for validation.

val predictions_2 = model_2.transform(test)
    .select("features", "final_status", "predictions")

val f1_score_2 = evaluator.evaluate(predictions_2)

printf("The f1 score on test set is : %.3f\n", f1_score_2)

### variable engineering

we propose to categorize goal variable

In [ ]:
println(s"\tNombre de lignes : ${preprocessed.count}")
println(s"\tNombre de colonnes : ${preprocessed.columns.length}")

In [ ]:
def goalLog(x: Double): Double = {
    val x_out = log1p(x)
    x_out
}

val preprocessed_2: DataFrame = preprocessed
  .withColumn("goal", $"goal".cast("Double"))
  .filter($"goal" > 10.0)

println(s"\tNombre de lignes : ${preprocessed_2.count}")
println(s"\tNombre de colonnes : ${preprocessed_2.columns.length}")

In [ ]:
val Array(training_2, test_2) = preprocessed_2
    .randomSplit(Array(0.9, 0.1), 13)

In [ ]:
val splits = Array(Double.NegativeInfinity, 0.0, 10000.0, 100000.0, 1000000.0, Double.PositiveInfinity)

val goalBucketizer = new Bucketizer()
  .setInputCol("goal")
  .setOutputCol("goal_bucketed")
  .setSplits(splits)

val encoder_2 = new OneHotEncoderEstimator()
  .setInputCols(Array("country_indexed", "currency_indexed", "goal_bucketed"))
  .setOutputCols(Array("country_onehot", "currency_onehot", "goal_onehot"))

In [ ]:
val pipeline_2 = new Pipeline()
  .setStages(Array(tokenizer, remover, countVectorizer, idf,
                   countryIndexer, currencyIndexer,
                   encoder, assembler, lr))

In [ ]:
// create a model from training data
val model_2 = pipeline_2.fit(training_2)

In [ ]:
// Make predictions from test data
val dfWithSimplePredictions_2 = model_2.transform(test_2)
    .select("features", "final_status", "predictions")

dfWithSimplePredictions_2.groupBy("final_status", "predictions").count.show()

In [ ]:
val f1_score = evaluator.evaluate(dfWithSimplePredictions_2)

printf("The f1 score on test set is : %.3f\n", f1_score)

### Using another classifier algorithm

In [ ]:
val dt = new DecisionTreeClassifier()
  .setMaxDepth(3)
  .setFeaturesCol("features")
  .setLabelCol("label")
  .setPredictionCol("predictions")


Supprimer goal = 0 et goal 95%

stat.approximateQuantile